# Attacks in Mexico during elections

_I dataset sono stati prelevati dall'Harvard Dataverse – British Journal of Political Science Dataverse. The bibliography can be found at the end of this notebook._

The motivation for the dataset lies in a surprising wave of attacks characterized by high-profile violence and perpetrated by Mexican drug cartels. These attacks were directed against hundreds of political candidates between 2007 and 2012. 

The researchers suggest that the attacks were perpetrated in an attempt to establish criminal governance, demonstrating the vulnerability of Mexican politics.